# Can a multivariate regression fit help for LHF'?

In [9]:
from sklearn import linear_model
# Some calculation for the LHF coupling and separation across spatial scales.
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import xarray as xr
import glob
import sys; sys.path.insert(1, '/home/alessandro/thesis_material/EURECA_scripts/')
import geometry as gm
import warnings; warnings.filterwarnings("ignore")
import re

In [10]:

avg_primes_ls = glob.glob('/venus/tesi_ale/data_EU/VARIANCE_LHF/daily/FEB/ForCouComparison/primes/*barsprimes*')
avg_primes_ls = sorted(avg_primes_ls)


primes_highpas, primes_inf, primes_bandpas = [], [], []

for i, item in enumerate(avg_primes_ls):
    if re.search(r'passalto', item):
        primes_highpas.append(item)
    elif re.search(r'sigma-inf', item):
        primes_inf.append(item)
    elif re.search(r'band-pass', item):
        primes_bandpas.append(item)

In [ ]:
name_vars = ['sst_bar', 'sst_prime', 'ta_bar', 'ta_prime', 'qa_bar', 
             'qa_prime', 'U_bar', 'U_prime', 'RH_bar', 'RH_prime', 'lhf_prime']

# Coupled model

## High-pass

In [11]:
ds_hp_primes_cou, ds_hp_primes_for = xr.open_dataset(primes_highpas[0]), xr.open_dataset(primes_highpas[1])   
  


In [26]:
ds_in_use = ds_hp_primes_cou


lhf_prime = ds_in_use['lhf_prime'].values.flatten()
lhf_prime = lhf_prime[~np.isnan(lhf_prime)]


sst_prime, U_prime, qa_prime = ds_in_use['sst_prime'].values, ds_in_use['U_prime'].values, ds_in_use['qa_prime'].values

sst_prime = sst_prime[~np.isnan(sst_prime)].flatten()
U_prime = U_prime[~np.isnan(U_prime)].flatten()
qa_prime = qa_prime[~np.isnan(qa_prime)].flatten()

primes = np.array([sst_prime, U_prime, qa_prime]).transpose()


In [27]:
regr = linear_model.LinearRegression()
fitting = regr.fit(primes, lhf_prime)

In [29]:
regr.score(primes, lhf_prime)

0.984033626522989

In [36]:
print(f"coeff sst' [K] = {round(fitting.coef_[0], 2)}")
print(f"coeff U' [m/s] = {round(fitting.coef_[1], 2)}")
print(f"coeff q' [kg/kg] = {round(fitting.coef_[2], 2)}")
print(f"intercept = {round(fitting.intercept_, 2)} W/m2")

coeff for sst' = 42.28
coeff for U' = 21.1
coeff for q' = -33213.23
intercept = 0.03 W/m2


# let's try to fit the variance with all the terms

In [43]:
lambda_vars_ls = glob.glob('/venus/tesi_ale/data_EU/VARIANCE_LHF/daily/FEB/ForCouComparison/lambda_vars/*')
lambda_vars_ls = sorted(lambda_vars_ls)

import re
indices = []
for i, item in enumerate(lambda_vars_ls):
    if re.search(r'lhfRH', item):
        print(f'index {i}')
        indices.append(i)

    # cleaning up which data to use
indices = np.array(indices).astype('int')
for index in sorted(indices, reverse=True):
    del lambda_vars_ls[index]
    
    
vars_highpas, vars_inf, vars_bandpas = [], [], []

for i, item in enumerate(lambda_vars_ls):
    if re.search(r'passalto', item):
        vars_highpas.append(item)
    elif re.search(r'sigma-inf', item):
        vars_inf.append(item)
    elif re.search(r'bandpass', item):
        vars_bandpas.append(item)

index 1
index 3
index 5
index 7
index 9
index 11


In [ ]:
['lambda_sst', 'lambda_U', 'lambda_ta', 'lambda_RH', 'lambda_q',
 'var_lhf', 'var_sst', 'var_U', 'var_ta', 'var_RH', 
 'covar_U_sst', 'covar_ta_sst', 'covar_RH_sst', 'covar_U_ta',
 'covar_U_RH', 'covar_ta_RH', 'covar_lhf_sst', 'covar_U_q', 'covar_q_sst', 'var_qa']

In [45]:
ds_in_use = xr.open_dataset(vars_highpas[0])


lambda_U = ds_in_use['lambda_U'].values
lambda_sst = ds_in_use['lambda_sst'].values
lambda_q = ds_in_use['lambda_q'].values

var_lhf = ds_in_use['var_lhf'].values
var_qa = ds_in_use['var_qa'].values
var_U = ds_in_use['var_U'].values
covar_U_q = ds_in_use['covar_U_q'].values

# sst terms
var_sst = ds_in_use['var_sst'].values
covar_U_sst = ds_in_use['covar_U_sst'].values
covar_q_sst = ds_in_use['covar_q_sst'].values



lambda_U = lambda_U[~np.isnan(lambda_U)].flatten()
lambda_sst = lambda_sst[~np.isnan(lambda_sst)].flatten()
lambda_q = lambda_q[~np.isnan(lambda_q)].flatten()

var_lhf = var_lhf[~np.isnan(var_lhf)].flatten()
var_qa = var_qa[~np.isnan(var_qa)].flatten()
var_U = var_U[~np.isnan(var_U)].flatten()
covar_U_q = covar_U_q[~np.isnan(covar_U_q)].flatten()

var_sst = var_sst[~np.isnan(var_sst)].flatten()
covar_U_sst = covar_U_sst[~np.isnan(covar_U_sst)].flatten()
covar_q_sst = covar_q_sst[~np.isnan(covar_q_sst)].flatten()



# sst_terms = (lambda_sst**2*var_sst + 2*lambda_sst*(lambda_U*covar_U_sst +\
#                                                                    lambda_q*covar_q_sst ))
# non_sst_terms = (lambda_U**2*var_U + lambda_q**2*var_qa + 2*lambda_q*lambda_U*covar_U_q)
# var_lhf_sum = sst_terms + non_sst_terms

In [46]:
X = np.array([var_sst, var_U, var_qa, covar_U_sst, covar_q_sst, covar_U_q]).transpose()

In [49]:
regr = linear_model.LinearRegression()
fitting = regr.fit(X, var_lhf)
print(f'R2 score for fit = {round(regr.score(X, var_lhf),2)}')

R2 score for fit = 0.92


In [50]:
print(f"coeff var_sst [K2] = {round(fitting.coef_[0], 2)}")
print(f"coeff var_U [m2/s2] = {round(fitting.coef_[1], 2)}")
print(f"coeff var_q [kg2/kg2] = {round(fitting.coef_[2], 2)}")
print(f"coeff covar_U_sst = {round(fitting.coef_[3], 2)}")
print(f"coeff covar_q_sst  = {round(fitting.coef_[4], 2)}")
print(f"coeff covar_U_q = {round(fitting.coef_[5], 2)}")

print(f"intercept = {round(fitting.intercept_, 2)} W2/m4")

coeff var_sst [K2] = 1638.89
coeff var_U [m2/s2] = 380.88
coeff var_q [kg2/kg2] = 1327493064.06
coeff covar_U_sst = 1960.33
coeff covar_q_sst  = -2414059.11
coeff covar_U_q = -1968801.47
intercept = -2.47 W2/m4


### estimate Standard Error over coefficients
from here: https://stackoverflow.com/questions/22381497/python-scikit-learn-linear-model-parameter-standard-error
https://stats.stackexchange.com/questions/405429/uncertainty-in-linear-regression-coefficients

In [61]:
N = len(var_lhf)
p = X.shape[1]+1

var_lhf_pred = fitting.predict(X)
res = var_lhf - var_lhf_pred
res_sum_of_squares = res.T @ res
sigma_sqrd = res_sum_of_squares / (N - p)   #[0, 0]


X_with_intercept = np.empty(shape=(N, p), dtype=np.float)
X_with_intercept[:, 0] = 1
X_with_intercept[:, 1:p] = X  #.values


var_beta_hat = np.linalg.inv(X_with_intercept.T @ X_with_intercept) * sigma_sqrd
for p_ in range(p):
    standard_error = var_beta_hat[p_, p_] ** 0.5
    print(f"SE(beta_hat[{p_}]): {standard_error}")

SE(beta_hat[0]): 0.03669934734935445
SE(beta_hat[1]): 0.6109827783105465
SE(beta_hat[2]): 0.3293733921311048
SE(beta_hat[3]): 555816.1535398214
SE(beta_hat[4]): 1.4860325175403253
SE(beta_hat[5]): 2197.4162076830794
SE(beta_hat[6]): 716.8051876739343


In [60]:
res_sum_of_squares

7763412867.758364

## Bandpass

In [ ]:
ds_bp_primes_cou, ds_bp_primes_for = xr.open_dataset(primes_bandpas[0]), xr.open_dataset(primes_bandpas[1]) 

In [37]:
ds_in_use = ds_bp_primes_cou

lhf_prime = ds_in_use['lhf_prime'].values.flatten()
lhf_prime = lhf_prime[~np.isnan(lhf_prime)]


sst_prime, U_prime, qa_prime = ds_in_use['sst_prime'].values, ds_in_use['U_prime'].values, ds_in_use['qa_prime'].values

sst_prime = sst_prime[~np.isnan(sst_prime)].flatten()
U_prime = U_prime[~np.isnan(U_prime)].flatten()
qa_prime = qa_prime[~np.isnan(qa_prime)].flatten()

primes = np.array([sst_prime, U_prime, qa_prime]).transpose()


In [38]:
regr = linear_model.LinearRegression()
fitting = regr.fit(primes, lhf_prime)
print(f'R2 score for fit = {regr.score(primes, lhf_prime)}')

R2 score for fit = 0.985025913492125


In [39]:
print(f"coeff sst' [K] = {round(fitting.coef_[0], 2)}")
print(f"coeff U' [m/s] = {round(fitting.coef_[1], 2)}")
print(f"coeff q' [kg/kg] = {round(fitting.coef_[2], 2)}")
print(f"intercept = {round(fitting.intercept_, 2)} W/m2")

coeff sst' [K] = 42.36
coeff U' [m/s] = 21.48
coeff q' [kg/kg] = -33325.79
intercept = 0.02 W/m2
